In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)

In [3]:
net(X)

tensor([[-0.1539,  0.2439,  0.1419,  0.1061,  0.2851, -0.0341, -0.0266,  0.1835,
          0.0846, -0.1791],
        [-0.0586,  0.1759,  0.1577,  0.0864,  0.2482,  0.0832, -0.0386,  0.1961,
          0.0206, -0.2415]], grad_fn=<AddmmBackward0>)

In [4]:
'''
F.relu()是函数调用，一般使用在foreward函数。
nn.ReLU()是模块调用，一般在定义网络层的时候使用。
当用print(net)输出时，会有nn.ReLU()层，而F.ReLU()是没有输出的。
'''
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[-0.1219,  0.2362, -0.0249, -0.2978, -0.0079, -0.0140,  0.0063,  0.0402,
          0.1832,  0.0780],
        [-0.1683,  0.2141, -0.0024, -0.2245,  0.0317, -0.0455,  0.0575, -0.0456,
          0.2931,  0.1748]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, block in enumerate(args):
            self._modules[str(idx)] = block
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2353, -0.0136,  0.2088, -0.0389,  0.0985, -0.0635,  0.0584, -0.1806,
         -0.0005, -0.2087],
        [-0.1169,  0.0521,  0.1133, -0.0147,  0.0965, -0.0599,  0.1453, -0.1749,
          0.0172, -0.1124]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 0-1均匀分布
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0030, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

In [11]:
net = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
net(X)

tensor(0.1826, grad_fn=<SumBackward0>)